# Chapter 22: Forms and JavaScript

---

## Introduction

Forms are the primary method for collecting user input on the web. Whether it's a login form, a registration page, a search box, or a complex multi‑step checkout, forms are everywhere. While HTML provides the structure for forms (as covered in Chapter 7), JavaScript brings them to life by enabling validation, dynamic behavior, and a smoother user experience.

In this chapter, you will learn how to:

- Access and manipulate form elements from JavaScript.
- Read and set values for different input types.
- Validate form data using both custom logic and the browser’s built‑in Constraint Validation API.
- Handle form submission, including sending data asynchronously with AJAX.
- Provide real‑time feedback as the user types.
- Work with special inputs like checkboxes, radio buttons, select dropdowns, and file inputs.

By the end of this chapter, you will be able to create robust, user‑friendly forms that validate data before it ever reaches your server.

---

## 22.1 Accessing Form Elements

Before you can work with a form in JavaScript, you need to access its elements. The DOM provides several ways to do this.

### The `forms` Collection

Every form in a document is automatically added to the `document.forms` collection. You can access a specific form by its index (in source order) or by its `name` or `id` attribute.

```html
<form name="login" id="login-form">
    <input type="text" name="username">
    <input type="password" name="password">
    <button type="submit">Log In</button>
</form>
```

```javascript
// By index (first form on the page)
const form1 = document.forms[0];

// By name
const form2 = document.forms.login;      // using name="login"
const form3 = document.forms['login'];    // same

// By id (not directly through forms, but you can use getElementById)
const form4 = document.getElementById('login-form');
```

### Accessing Form Controls with `elements`

Once you have a reference to the form, the `elements` property gives you access to all its controls (inputs, buttons, etc.) in an array‑like collection. You can access them by index, by name, or by id.

```javascript
const form = document.forms.login;

// By index
const firstInput = form.elements[0];   // username input

// By name (most common)
const usernameInput = form.elements.username;   // or form.username
const passwordInput = form.elements.password;   // or form.password

// By id (if the element has an id)
const usernameInput = form.elements['username']; // same
```

**Note:** Using `form.elementName` (e.g., `form.username`) is a convenient shorthand, but it may conflict with existing form properties. The safer approach is to use `form.elements.elementName` or `form.elements['elementName']`.

### Iterating Over Form Controls

You can loop through all form elements:

```javascript
const form = document.forms[0];
for (let i = 0; i < form.elements.length; i++) {
    const element = form.elements[i];
    console.log(element.name, element.type, element.value);
}
```

This is useful for resetting, validating, or serialising all fields.

---

## 22.2 Working with Input Values

Different input types store their values in different properties. The most common is the `value` property, but checkboxes and radio buttons use the `checked` property.

### Text‑Based Inputs

For `<input type="text">`, `<textarea>`, `<input type="password">`, `<input type="email">`, etc., use the `value` property.

```javascript
const username = document.forms.login.username;
console.log(username.value);        // get current value
username.value = 'prefilled';        // set value
```

### Checkboxes and Radio Buttons

Checkboxes have a `checked` boolean property. Radio buttons within the same `name` group are mutually exclusive; you can check which one is selected by iterating or using the `:checked` pseudo‑class in a selector.

```html
<input type="checkbox" name="subscribe" id="subscribe" checked>
<label for="subscribe">Subscribe to newsletter</label>

<input type="radio" name="gender" value="male" id="male">
<label for="male">Male</label>
<input type="radio" name="gender" value="female" id="female" checked>
<label for="female">Female</label>
```

```javascript
// Checkbox
const subscribe = document.getElementById('subscribe');
console.log(subscribe.checked);   // true or false
subscribe.checked = false;        // uncheck

// Radio buttons - get the selected value
const genderInputs = document.querySelectorAll('input[name="gender"]');
let selectedGender;
for (const radio of genderInputs) {
    if (radio.checked) {
        selectedGender = radio.value;
        break;
    }
}
console.log(selectedGender);   // "female"

// Or using the :checked selector
const checkedRadio = document.querySelector('input[name="gender"]:checked');
console.log(checkedRadio ? checkedRadio.value : 'none selected');
```

### Select Dropdowns

A `<select>` element has options accessible via the `options` collection. The selected option(s) can be read using `selectedIndex` (for single select) or by iterating through `options` and checking the `selected` property.

```html
<select name="country" id="country">
    <option value="us">United States</option>
    <option value="ca" selected>Canada</option>
    <option value="mx">Mexico</option>
</select>
```

```javascript
const select = document.getElementById('country');
// Get selected value
console.log(select.value);               // "ca"
// Get selected index
console.log(select.selectedIndex);        // 1 (zero-based)
// Access selected option object
const selectedOption = select.options[select.selectedIndex];
console.log(selectedOption.text);         // "Canada"
```

For multi‑select (`<select multiple>`), you need to iterate:

```javascript
const multiSelect = document.getElementById('languages');
const selectedValues = [];
for (const option of multiSelect.options) {
    if (option.selected) {
        selectedValues.push(option.value);
    }
}
```

### File Inputs

File inputs hold a `FileList` object in their `files` property. Each file in the list has properties like `name`, `size`, `type`, and `lastModified`.

```html
<input type="file" name="avatar" id="avatar" accept="image/*">
```

```javascript
const fileInput = document.getElementById('avatar');
fileInput.addEventListener('change', (event) => {
    const files = event.target.files;
    if (files.length > 0) {
        const file = files[0];
        console.log('File name:', file.name);
        console.log('File size:', file.size, 'bytes');
        console.log('File type:', file.type);
    }
});
```

We'll explore file previews later in this chapter.

---

## 22.3 Form Validation with JavaScript

Client‑side validation improves user experience by providing immediate feedback, but **never trust client‑side validation alone**—always validate on the server as well.

### Simple Custom Validation

You can write your own validation functions that check the input values and display error messages.

```html
<form id="register">
    <div>
        <label for="email">Email:</label>
        <input type="email" id="email" name="email">
        <span class="error" id="emailError"></span>
    </div>
    <div>
        <label for="password">Password:</label>
        <input type="password" id="password" name="password">
        <span class="error" id="passwordError"></span>
    </div>
    <div>
        <label for="confirm">Confirm Password:</label>
        <input type="password" id="confirm" name="confirm">
        <span class="error" id="confirmError"></span>
    </div>
    <button type="submit">Register</button>
</form>
```

```javascript
const form = document.getElementById('register');
form.addEventListener('submit', (event) => {
    // Clear previous errors
    document.querySelectorAll('.error').forEach(el => el.textContent = '');

    let isValid = true;

    const email = document.getElementById('email');
    const password = document.getElementById('password');
    const confirm = document.getElementById('confirm');

    // Email validation (simple presence check)
    if (!email.value) {
        document.getElementById('emailError').textContent = 'Email is required.';
        isValid = false;
    }

    // Password length check
    if (password.value.length < 6) {
        document.getElementById('passwordError').textContent = 'Password must be at least 6 characters.';
        isValid = false;
    }

    // Confirm password match
    if (password.value !== confirm.value) {
        document.getElementById('confirmError').textContent = 'Passwords do not match.';
        isValid = false;
    }

    if (!isValid) {
        event.preventDefault(); // Prevent form submission
    }
    // If isValid is true, the form submits normally (could also submit via AJAX)
});
```

### The Constraint Validation API

Modern browsers provide a built‑in validation API that works with HTML5 validation attributes (`required`, `pattern`, `min`, `max`, etc.). You can check an element's validity using the `checkValidity()` method and inspect the `validity` object for detailed error types.

**Key properties of the `validity` object:**

| Property          | Description |
|-------------------|-------------|
| `valueMissing`    | True if the element has the `required` attribute but no value. |
| `typeMismatch`    | True when the value does not match the expected type (e.g., `email` or `url`). |
| `patternMismatch` | True when the value does not match the `pattern` regex. |
| `tooLong`         | True when the value exceeds the `maxlength` attribute. |
| `tooShort`        | True when the value is shorter than the `minlength` attribute. |
| `rangeUnderflow`  | True when the value is less than the `min` attribute. |
| `rangeOverflow`   | True when the value is greater than the `max` attribute. |
| `stepMismatch`    | True when the value does not fit the `step` attribute. |
| `badInput`        | True when the browser cannot convert the value (e.g., "abc" for a number input). |
| `customError`     | True if a custom validity message has been set via `setCustomValidity()`. |
| `valid`           | True if all constraints are satisfied. |

You can also set a custom validity message with `setCustomValidity(message)`. When you set a message, the element becomes invalid until you clear it by calling `setCustomValidity('')`.

```javascript
const email = document.getElementById('email');
email.addEventListener('input', () => {
    if (email.validity.typeMismatch) {
        email.setCustomValidity('Please enter a valid email address.');
    } else {
        email.setCustomValidity(''); // clear any previous message
    }
});
```

**Using `checkValidity()` and `reportValidity()`:**

- `element.checkValidity()` returns `true` if the element satisfies all constraints, otherwise `false`.
- `element.reportValidity()` does the same but also shows the browser’s native validation UI (the tooltip) for the first invalid field.

You can combine the API with custom error displays to maintain consistent styling.

```javascript
form.addEventListener('submit', (event) => {
    if (!form.checkValidity()) {
        event.preventDefault();
        // Optionally, display custom error messages
        // Or rely on reportValidity() to show native bubbles
        form.reportValidity();
    }
});
```

### Example: Password Confirmation with Constraint Validation

The Constraint Validation API does not natively support password‑match checks, but you can use `setCustomValidity` to integrate it.

```javascript
const password = document.getElementById('password');
const confirm = document.getElementById('confirm');

function checkPasswordMatch() {
    if (password.value !== confirm.value) {
        confirm.setCustomValidity('Passwords do not match.');
    } else {
        confirm.setCustomValidity('');
    }
}

password.addEventListener('input', checkPasswordMatch);
confirm.addEventListener('input', checkPasswordMatch);

form.addEventListener('submit', (event) => {
    checkPasswordMatch(); // ensure latest state
    if (!form.checkValidity()) {
        event.preventDefault();
        form.reportValidity();
    }
});
```

Now, if passwords don’t match, the browser will show a custom error bubble when the form is submitted.

---

## 22.4 Handling Form Submission

By default, submitting a form sends an HTTP request (usually GET or POST) to the URL specified in the `action` attribute, causing the page to reload. With JavaScript, you can intercept the submission to:

- Validate data before sending.
- Send data asynchronously (AJAX) without reloading the page.
- Show a loading spinner or success message.

### The `submit` Event

Attach an event listener to the form’s `submit` event. To prevent the default browser behaviour (page reload), call `event.preventDefault()`.

```javascript
const form = document.getElementById('myForm');
form.addEventListener('submit', (event) => {
    event.preventDefault(); // Stop the default submission

    // Perform validation, gather data, send AJAX, etc.
});
```

### AJAX Form Submission

AJAX (Asynchronous JavaScript and XML) lets you send data to the server and process the response without leaving the page. Today, the `fetch()` API is the modern standard.

```javascript
form.addEventListener('submit', async (event) => {
    event.preventDefault();

    // Collect form data (using FormData API)
    const formData = new FormData(form);

    // Optional: Convert to plain object (if server expects JSON)
    const data = Object.fromEntries(formData.entries());

    try {
        const response = await fetch('/api/submit', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify(data),
        });

        if (response.ok) {
            const result = await response.json();
            console.log('Success:', result);
            // Show success message to user
        } else {
            console.error('Server error:', response.statusText);
            // Show error message
        }
    } catch (error) {
        console.error('Network error:', error);
        // Show network error
    }
});
```

**Note:** For file uploads, you cannot use `application/json`; you must use `multipart/form-data`. In that case, pass the `FormData` object directly as the body (do not set `Content-Type`; the browser will set it automatically with the correct boundary).

```javascript
const response = await fetch('/upload', {
    method: 'POST',
    body: formData  // FormData handles multipart encoding
});
```

---

## 22.5 Real‑Time Validation

Real‑time validation (validation as the user types) provides immediate feedback, which can greatly improve the user experience. However, it must be implemented carefully to avoid annoying the user (e.g., showing errors before they finish typing).

### Input Events

- `input` – fires every time the value changes (typing, pasting, etc.). Best for live validation.
- `change` – fires after the input loses focus and its value has changed. Good for “final” validation.
- `blur` – fires when the input loses focus, regardless of change. Useful for validating after the user leaves the field.

```javascript
const email = document.getElementById('email');
const emailError = document.getElementById('emailError');

email.addEventListener('input', () => {
    if (email.validity.valid) {
        emailError.textContent = '';
        email.classList.remove('invalid');
        email.classList.add('valid');
    } else {
        emailError.textContent = 'Please enter a valid email address.';
        email.classList.remove('valid');
        email.classList.add('invalid');
    }
});
```

### Debouncing for Performance

If your validation involves expensive operations (e.g., making an API call to check username availability), you should debounce the event handler so it doesn’t run on every keystroke. Debouncing waits for a pause in typing before executing.

```javascript
function debounce(func, delay) {
    let timeout;
    return function (...args) {
        clearTimeout(timeout);
        timeout = setTimeout(() => func.apply(this, args), delay);
    };
}

const checkUsernameAvailability = debounce(async (event) => {
    const username = event.target.value;
    if (username.length < 3) return;
    // Make API call
    const response = await fetch(`/api/check-username?username=${username}`);
    const available = await response.json();
    // Update UI
}, 500);

usernameInput.addEventListener('input', checkUsernameAvailability);
```

### User Feedback Patterns

Provide clear, accessible feedback:

- **Inline error messages** next to the field.
- **Visual cues** – green borders for valid fields, red for invalid.
- **Icons** – checkmark, exclamation triangle.
- **Live summaries** – a list of errors that updates as the user corrects them.

Always ensure that error messages are announced by screen readers (e.g., using `aria-live` regions or `role="alert"`).

---

## 22.6 Working with Special Inputs

### Checkboxes and Radio Groups

Handling groups of checkboxes (e.g., for selecting multiple hobbies) often involves listening for changes and collecting selected values.

```html
<fieldset>
    <legend>Choose your hobbies:</legend>
    <label><input type="checkbox" name="hobby" value="reading"> Reading</label>
    <label><input type="checkbox" name="hobby" value="music"> Music</label>
    <label><input type="checkbox" name="hobby" value="sports"> Sports</label>
</fieldset>
<p>Selected: <span id="selectedHobbies"></span></p>
```

```javascript
const checkboxes = document.querySelectorAll('input[name="hobby"]');
const selectedSpan = document.getElementById('selectedHobbies');

function updateSelected() {
    const selected = Array.from(checkboxes)
        .filter(cb => cb.checked)
        .map(cb => cb.value)
        .join(', ');
    selectedSpan.textContent = selected || 'None';
}

checkboxes.forEach(cb => cb.addEventListener('change', updateSelected));
```

### Select Dropdowns (Single and Multiple)

For a single select, listening to the `change` event gives you the selected value directly. For multi‑select, you need to iterate through the options.

```html
<select id="car" name="car">
    <option value="volvo">Volvo</option>
    <option value="bmw">BMW</option>
    <option value="audi">Audi</option>
</select>

<select id="colors" name="colors" multiple>
    <option value="red">Red</option>
    <option value="green">Green</option>
    <option value="blue">Blue</option>
</select>
```

```javascript
// Single select
const carSelect = document.getElementById('car');
carSelect.addEventListener('change', (e) => {
    console.log('Selected car:', e.target.value);
});

// Multi-select
const colorSelect = document.getElementById('colors');
colorSelect.addEventListener('change', () => {
    const selected = Array.from(colorSelect.selectedOptions).map(opt => opt.value);
    console.log('Selected colors:', selected);
});
```

### File Inputs and FileReader

File inputs allow users to upload files. The `FileReader` API lets you read file contents (e.g., for image previews) without sending the file to the server.

```html
<input type="file" id="avatar" accept="image/*">
<img id="preview" src="#" alt="Preview" style="display: none; max-width: 200px;">
```

```javascript
const fileInput = document.getElementById('avatar');
const preview = document.getElementById('preview');

fileInput.addEventListener('change', (event) => {
    const file = event.target.files[0];
    if (file) {
        const reader = new FileReader();
        reader.onload = (e) => {
            preview.src = e.target.result;
            preview.style.display = 'block';
        };
        reader.readAsDataURL(file); // reads as data URL (base64)
    } else {
        preview.src = '#';
        preview.style.display = 'none';
    }
});
```

**Other uses of FileReader:** `readAsText()` for reading text files, `readAsArrayBuffer()` for binary data.

### Date Inputs

HTML5 date inputs (`<input type="date">`) return a string in `YYYY-MM-DD` format. You can convert this to a JavaScript `Date` object if needed.

```javascript
const dateInput = document.getElementById('birthdate');
dateInput.addEventListener('change', () => {
    const dateString = dateInput.value; // "2024-05-20"
    if (dateString) {
        const date = new Date(dateString + 'T00:00:00'); // add time to avoid timezone issues
        console.log(date.toLocaleDateString());
    }
});
```

You can also set the input’s value using the same format:

```javascript
const today = new Date().toISOString().split('T')[0];
dateInput.value = today;
```

---

## Chapter Summary

In this chapter, you learned how to harness JavaScript to create dynamic, user‑friendly forms:

- **Accessing form elements** – using `document.forms`, `form.elements`, and direct references by name.
- **Reading and writing values** – for text inputs, checkboxes, radio buttons, selects, and files.
- **Validation** – both custom validation and the powerful Constraint Validation API.
- **Handling submission** – intercepting the submit event, preventing default, and sending data via AJAX with `fetch()`.
- **Real‑time validation** – using input events and debouncing to provide instant feedback.
- **Special inputs** – working with groups, multi‑selects, and file previews with the FileReader API.

### Key Takeaways

- Always validate on the server; client‑side validation is for user experience only.
- The Constraint Validation API gives you fine‑grained control and integrates with HTML5 validation attributes.
- Use `preventDefault()` on the submit event to stop page reloads when using AJAX.
- Real‑time validation should be helpful, not intrusive—consider debouncing for expensive checks.
- File inputs require special handling; the FileReader API enables client‑side previews.

### Practice Exercises

1. Create a sign‑up form with fields for name, email, password, and password confirmation. Implement real‑time validation that shows/hides error messages and uses the Constraint Validation API where appropriate. Disable the submit button until all fields are valid.

2. Build a “Add/Remove Hobbies” interface: a set of checkboxes and a text input that allows the user to add new custom hobbies. Display the list of selected hobbies in real time.

3. Enhance a file input to show a preview of the selected image and display file size and type. Also provide a button to clear the selection.

4. Create a multi‑step form (e.g., shipping and payment) that uses JavaScript to show one step at a time. Validate each step before moving to the next.

5. Write a function that serialises a form into a JavaScript object (key‑value pairs) and another that populates a form from such an object. Test it with various input types.

---

## Coming Up Next

**Chapter 23: Asynchronous JavaScript**

In the next chapter, we’ll dive deep into asynchronous programming—a crucial skill for working with AJAX, timers, and browser APIs. You’ll learn about callbacks, Promises, async/await, the Fetch API, and more, building on the AJAX concepts introduced here.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='21. events_in_javascript.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='23. asynchronous_javascript.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
